In [3]:
!pip install pdfminer.six
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install python-dateutil


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 87.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
import re
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import string
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pdfminer.high_level import extract_text

In [5]:
# Fungsi untuk ekstraksi teks dari PDF
def extract_text_from_pdf(pdf_path):
    try:
        text = extract_text(pdf_path)
        return text
    except Exception as e:
        return str(e)

In [6]:
# Fungsi untuk preprocessing teks
def clean_text(text):
    nlp = spacy.load("en_core_web_sm")

    # Memastikan bahwa karakter "-" dihapus, serta karakter lainnya yang tidak relevan
    text = re.sub(r"[^a-zA-Z0-9\s,:]", "", text)  # Menghapus karakter selain alfabet, angka, spasi, koma, titik dua, dan bullet

    doc = nlp(text)

    # Proses pembersihan teks untuk stop words
    cleaned_text = []
    for token in doc:
        if token.text.lower() not in STOP_WORDS:
            cleaned_text.append(token.text.lower())

    return " ".join(cleaned_text)

In [16]:
pdf_path = "er.pdf"  # Replace with your PDF file path
extracted_text = extract_text_from_pdf(pdf_path)
cleaned_text = clean_text(extracted_text)
print(cleaned_text)

muhammad erlangga prasetya 
 6281293603711   2110631170078studentunsikaacid   https : wwwlinkedincominmuhammaderlanggaprasetya   https : 
 githubcomerlanggsss     https : bitlyportofoliomerlanggaprstya 

 bogor , indonesia 

 year informatics student university singaperbangsa karawang actively involved campus 
 organizational activities , interest field , high commitment learning trying interesting things life , 
 deep understanding concepts cloud computing , data design strong analytical practical skills fast learner , 
 ambitious able compete professional environment participate technological advances 

 education level 

 university singaperbangsa karawang   karawang , indonesia 

 sep 2021   jul 2025 expected 

 bachelor degree informatics , 378400 

 work experiences 

 msib 5   bangkit academy 2023 google , goto , traveloka 

 aug 2023   jan 2024 

 cloud computing learning path 

 successfully graduated distinction cohort 
 develop skills google cloud platform , cloud infrastruc

In [8]:
# Personal Info
def extract_personal_info(raw_text):
    phone = re.search(r'(\+62\s?|\(?\+62\)?|\s?0)8\d{10,13}', raw_text)
    email = re.search(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', raw_text)
    linkedin = re.search(r'(?:https?://)?(?:www\.)?linkedin\.com/\S+', raw_text)
    github = re.search(r'(?:https?://)?(?:www\.)?github\.com/\S+', raw_text)
    degree = re.search(r'Bachelor\s+(?:degree|of)\s+[A-Za-z\s]+', raw_text, re.IGNORECASE)

    # Get name from first non-empty line
    lines = [line.strip() for line in raw_text.split("\n") if line.strip()]
    name = lines[0] if lines else None

    return {
        "Phone Number": phone.group(0) if phone else None,
        "Email": email.group(0) if email else None,
        "Github": github.group(0) if github else None,
        "LinkedIn": linkedin.group(0) if linkedin else None,
        "Degree": degree.group(0) if degree else None
    }

In [9]:
def extract_skills(cleaned_text):
    # List of tech skills categories
    tech_skills = {
        "Programming Languages": [
            "python", "java", "javascript", "typescript", "c\\+\\+", "c#",
            "ruby", "php", "swift", "kotlin", "go", "rust", "scala"
        ],
        "Web Developer": [
            "html", "css", "react", "angular", "vue", "nodejs", "django",
            "flask", "spring", "laravel", "express", "graphql", "rest api"
        ],
        "Mobile Development": [
            "android", "ios", "flutter", "react native", "kotlin", "swift",
            "xamarin", "ionic", "jetpack compose", "android studio"
        ],
        "Cloud & DevOps": [
            "aws", "azure", "google cloud", "docker", "kubernetes",
            "jenkins", "terraform", "ansible", "cloud computing",
            "saas", "iaas", "paas"
        ],
        "Data & AI": [
            "machine learning", "data science", "deep learning", "tensorflow",
            "pytorch", "pandas", "numpy", "scikit-learn", "data analysis",
            "big data", "spark", "hadoop", "sql", "nosql", "tableau",
            "artificial intelligence", "computer vision",
            "natural language processing","nlp","ocr",
            "seaborn", "data engineering", "data visualization "
        ],
        "Design & UX": [
            "ui", "ux", "figma", "sketch", "adobe xd", "photoshop",
            "user interface", "user experience", "graphic design"
        ],
        "Databases": [
            "mysql", "postgresql", "mongodb", "sqlite", "redis",
            "oracle", "cassandra", "firebase"
        ],
        "Other Technologies": [
            "git", "linux", "blockchain", "cybersecurity", "network security",
            "agile", "scrum", "microservices", "serverless"
        ]
    }

    # Flatten the skills list for easier matching
    all_skills = [skill for category in tech_skills.values() for skill in category]

    extracted_skills = []

    # Try different skill extraction patterns
    for pattern in [r'(?:Skills?:|Technical\s+Skills?:)\s*(.*?)(?=\n\n|\n[A-Z]|$)', r'Proficient\s+in:?\s*(.*?)(?=\n\n|\n[A-Z]|$)']:
        match = re.search(pattern, cleaned_text, re.IGNORECASE | re.DOTALL)
        if match:
            skills_text = match.group(1).lower()

            # Find matching skills
            for skill in all_skills:
                if re.search(r'\b' + re.escape(skill.lower()) + r'\b', skills_text):
                    extracted_skills.append(skill)

    # If no skills found via sections, do a comprehensive scan
    if not extracted_skills:
        for skill in all_skills:
            if re.search(r'\b' + re.escape(skill.lower()) + r'\b', cleaned_text.lower()):
                extracted_skills.append(skill)

    # Remove duplicates and sort
    extracted_skills = sorted(set(extracted_skills))

    return extracted_skills

In [10]:
def extract_work_experience(raw_text):
    work_experiences = []

    # Find the work experience section
    exp_match = re.search(r'Work\s+Experiences?:?\s*(.*?)(?=\n\s*\n|\n[A-Z]|$)', cleaned_text, re.DOTALL | re.IGNORECASE)
    if exp_match:
        exp_text = exp_match.group(1).strip()

        # Split the work experience section by job
        jobs = exp_text.split('\n\n')

        for job in jobs:
            job = job.strip()
            if not job:  # Skip empty entries
                continue

            # Extract job name (first line)
            job_lines = job.split('\n')
            job_name = job_lines[0].strip()  # First line is the job name

            # Extract duration (assumed to be the second line)
            duration_match = re.search(r'(\w+\s+\d{4})\s*-\s*(\w+\s+\d{4})', job)
            duration_text = duration_match.group(0) if duration_match else ""
            duration = parse_work_duration(duration_text)

            # Extract description (everything after the duration)
            description_matches = re.findall(r'•\s*(.+)|-\s*(.+)', job)
            descriptions = [desc[0] or desc[1] for desc in description_matches if desc[0] or desc[1]]

            # If descriptions are not found, try to capture remaining lines
            if not descriptions:
                descriptions = [line.strip() for line in job_lines[2:] if line.strip()]

            # Clean up descriptions to ensure they are meaningful
            descriptions = [desc for desc in descriptions if len(desc) > 10]

            work_exp = {
                "Job Name": job_name,
                "Description": descriptions,
                "Duration": str(duration)
            }

            work_experiences.append(work_exp)

    return work_experiences if work_experiences else [{
        "Job Name": "No Experience Found",
        "Description": [],
        "Duration": "0"
    }]

In [11]:
def extract_certifications(cleaned_text):
    cert_patterns = [
        r'Certifications?:?\s*(.*?)(?=\n\n|\n[A-Z]|$)',
        r'Certificates?:?\s*(.*?)(?=\n\n|\n[A-Z]|$)'
    ]

    certifications = []
    for pattern in cert_patterns:
        matches = re.findall(pattern, cleaned_text, re.IGNORECASE | re.DOTALL)

        for match in matches:
            match = match.replace("\n", " ")

            certifications.extend([re.sub(r'\s+', ' ', cert.strip()) for cert in match.split(',') if cert.strip()])

    return list(set(certifications))


In [12]:
# Function to parse cleaned CV data to JSON format
def parse_cv_to_json(raw_text, cleaned_text):
    lines = [line.strip() for line in cleaned_text.split("\n") if line.strip()]
    name = lines[0] if lines else "Unknown"

    return {
        name: {
            "Personal Info": extract_personal_info(raw_text),
            "Skills": extract_skills(cleaned_text),
            "Work Experience": extract_work_experience(raw_text),
            "Certification": extract_certifications(cleaned_text)
        }
    }

In [14]:
def process_cv(pdf_path):
    # Ekstraksi teks dari PDF
    extracted_text = extract_text_from_pdf(pdf_path)

    # Pembersihan teks
    cleaned_text = clean_text(extracted_text)

    # Memastikan bahwa data yang telah dibersihkan dan data asli diteruskan ke fungsi ekstraksi
    result = parse_cv_to_json(extracted_text, cleaned_text)
    return json.dumps(result, indent=4, ensure_ascii=False)

In [15]:
result = process_cv("er.pdf")
print(result)

{
    "muhammad erlangga prasetya": {
        "Personal Info": {
            "Phone Number": "+6281293603711",
            "Email": "2110631170078@student.unsika.ac.id",
            "Github": "github.com/erlanggsss",
            "LinkedIn": "https://www.linkedin.com/in/muhammaderlanggaprasetya/",
            "Degree": "Bachelor Degree in Informatics"
        },
        "Skills": [
            "android",
            "cloud computing",
            "figma",
            "flask",
            "google cloud",
            "javascript",
            "nodejs",
            "python"
        ],
        "Work Experience": [
            {
                "Job Name": "No Experience Found",
                "Description": [],
                "Duration": "0"
            }
        ],
        "Certification": [
            "elastic google cloud infrastructure : scaling automation gcsb",
            "google cloud fundamentals : core infrastructure gcsb",
            "essential google cloud infrastructure : c